In [1]:
import pandas as pd
import math
from urllib.parse import unquote
from collections import deque  

In [2]:

def title_parse(title):
    title = unquote(title)
    return title


In [3]:
columns = ['hashedIpAddress','timestamp','durationInSec','path','rating']
df = pd.read_csv('wikispeedia_paths-and-graph/paths_finished.tsv',delimiter='\t',encoding='UTF-8',skiprows=16,header=None,names=columns)
df['path']= df.path.apply(title_parse)
df=df.drop([2395])
df

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,"Yarralumla,_Australian_Capital_Territory;Austr...",1.0


In [4]:
columns = ['category','ID']
df2 = pd.read_csv('category-ids.csv',names=columns)

categories = list(df2['ID'])
paths = [0]*len(categories)
visited = [0]*len(categories)
shortest_paths = [0]*len(categories)
shortest_visited = [0]*len(categories)

In [5]:

columns = ['article','ID']
df3 = pd.read_csv('article-ids.csv',names=columns)
df3['article'] = df3.article.apply(title_parse)

art_ID = dict()

data = df3['article']
data_ID = df3['ID']

for i in range(len(data)):
    art_ID[data[i]]=data_ID[i]
       

In [6]:
art_category = dict()

data = open('article-categories2.csv', 'r') 
for line in data:
    line = line.split(',')
    
    for i in range(len(line)):
        if i==0:
            if line[i] not in art_category.keys():
                art_category[line[i]]=list()
        else:
            art_category[line[0]].append(line[i].strip())

data.close()

In [7]:

for index, row in df.iterrows():
    
    path_ = row['path']
    path_ = path_.split(';')
    
    queue = deque()
    
    for i in path_:
        if i!='<':
            queue.append(i)
        else:
            queue.pop()
    
    path_ = list(queue)
        
    
    
    if len(path_)>=2:
        temp = {}
        for art in path_:

            art_id = art_ID[str(art)]

            for cat in art_category[art_id]:

                cat_i=int(cat[1:])-1
                if cat not in temp.keys():
                    temp[cat]=0
                    paths[cat_i]=paths[cat_i]+1
                    temp[cat]=1

                visited[cat_i]=visited[cat_i]+1
        
        
        s=path_[0]
        d=path_[-1]
        temp = {}
    
    
        art_id_s = art_ID[str(s)]
        art_id_d = art_ID[str(d)]
        
        for cat in art_category[art_id_s]:
            cat_i=int(cat[1:])-1
            if cat not in temp.keys():
                temp[cat]=0
                shortest_paths[cat_i]=shortest_paths[cat_i]+1
                temp[cat]=1
                
            shortest_visited[cat_i]=shortest_visited[cat_i]+1
        
        for cat in art_category[art_id_d]:
            cat_i=int(cat[1:])-1
            if cat not in temp.keys():
                temp[cat]=0
                shortest_paths[cat_i]=shortest_paths[cat_i]+1
                temp[cat]=1
                
            shortest_visited[cat_i]=shortest_visited[cat_i]+1

In [8]:
df2 = pd.DataFrame(list(zip(categories,paths,visited,shortest_paths,shortest_visited)), columns =['categories', 'paths','visited','shortest_paths','shortest_visited']) 

In [9]:
df2.to_csv('category-paths.csv',index=False,header=None)